# 政大出國留學學校搜尋系統v2

### 想必想要出國交換的大家，看到國合處落落長的徵試學校清單，一定很難知道你/妳的成績可以去哪間學校吧!
### 於是我構想要推出一個方便大家搜尋的系統!

#### -----------------------目前功能-----------------------
#### (v2更新) 外語組的大家也可以用自己的語言成績/修課要求找到可以申請的學校了!

#### 英語組的大家，可以用自己的英文檢定成績或是GPA來找到可以申請的學校!

#### -----------------------未來更新-----------------------
#### 1. (英語組) 因為有些學校有另行規定細項分數，所以需要另外去分細項分數來供學生篩選!
#####    例: 荷蘭的葛洛賽恩大學除了要求TOEFL iBT 80 之外， Speaking 和 Writing 還各要求至少 19 分。

#### 2. (Bug修復) 會嘗試把下面的SettingWithCopyWarning及UserWarning處理掉(不影響運作但不美觀)。

#### 3. 再增加篩選條件! 首先會是交換資格及交換開始學期。

#### 4. 搜尋系統重新架構: 這個會等到所有的篩選條件都做好，再來思考怎麼搜尋比較方便。

### 資料來源: http://oic.nccu.edu.tw/oicweb4/listpartnerschool.php

## 整理資料

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
# 設好Column Names
column_names = ["國別","校名","甄試組別","限制錄取生交換出國時資格","締約學校交換申請截止時間",
                "語言成績要求","名額","交換期程","可交換開始學年度.學期（擇一）","獎學金補助", "費用","備註"]

In [3]:
# 讀入Data
df = pd.read_html("http://oic.nccu.edu.tw/oicweb4/listpartnerschool.php")
df = df[0]

In [4]:
#先檢視讀進來的Data
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,國別,校名,甄試組別,限制錄取生交換出國時資格,締約學校交換申請截止時間,語言成績要求,名額,交換期程,可交換開始學年度.學期（擇一）,獎學金補助,費用,備註
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,1學期或1年,109.01（一年） 109.01（一學期） 109.02（一學期）,NaN,學生出國期間依教務處註冊組學生出國選修課程實施辦法辦理學雜費等項目繳費，其餘費用自理或依兩校...,NaN
3,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,大二以上不含研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,1學期或1年,109.01（一年） 109.01（一學期） 109.02（一學期）,NaN,學生出國期間依教務處註冊組學生出國選修課程實施辦法辦理學雜費等項目繳費，其餘費用自理或依兩校...,NaN
4,馬來西亞,馬來西亞國立大學NATIONAL UNIVERSITY OF MALAYSIA,英語組,大二以上不含研究所,05-30 / 10-30,TOEFL iBT 80；GPA 3/4,2,1學期或1年,109.01（一年） 109.01（一學期） 109.02（一學期）,NaN,學生出國期間依教務處註冊組學生出國選修課程實施辦法辦理學雜費等項目繳費，其餘費用自理或依兩校...,NaN


In [5]:
# 資料清理: 將不重要的費用、交換期程、備註清除，將NaN值用"無資料"取代
df.columns = column_names
df.drop([0,1], axis = 0, inplace = True)
df.drop(["費用","交換期程","備註"], axis = 1, inplace = True)
df.replace(to_replace = np.NaN, value = "無資料", inplace = True)
df.reset_index(drop = True, inplace = True)

In [6]:
# 清理過後的資料
df

,國別,校名,甄試組別,限制錄取生交換出國時資格,締約學校交換申請截止時間,語言成績要求,名額,可交換開始學年度.學期（擇一）,獎學金補助
0,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
1,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組,大二以上不含研究所,06-30 / 11-30,TOEFL iBT 80或IELTS 6；GPA 3/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
2,馬來西亞,馬來西亞國立大學NATIONAL UNIVERSITY OF MALAYSIA,英語組,大二以上不含研究所,05-30 / 10-30,TOEFL iBT 80；GPA 3/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
3,香港,香港教育大學THE EDUCATION UNIVERSITY OF HONG KONG,英語組,大二以上不含研究所,04-20 / 10-15,TOEFL iBT 80或IELTS 6；GPA 2.7/4,1,109.01（一學期） 109.02（一學期）,無資料
4,香港,嶺南大學LINGNAN UNIVERSITY,英語組,大二以上不含研究所,04-30 / 10-31,TOEFL iBT 79或IELTS 6；GPA 2.5/4,2,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
...,...,...,...,...,...,...,...,...,...
332,以色列,海法大學UNIVERSITY OF HAIFA,英語組,大二以上不含研究所,05-15 / 11-15,TOEFL iBT 89,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
333,丹麥,南丹麥大學UNIVERSITY OF SOUTHERN DENMARK,英語組,研究所,04-01 / 10-01,TOEFL iBT 88或IELTS 6.5,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
334,丹麥,南丹麥大學UNIVERSITY OF SOUTHERN DENMARK,英語組,大二以上不含研究所,04-01 / 10-01,TOEFL iBT 88或IELTS 6.5,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
335,丹麥,奧胡斯大學AARHUS UNIVERSITY,英語組,大三以上不含研究所,05-01 / 11-01,TOEFL iBT 83或IELTS 6.5,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料


##  創建一個新的DataFrame (search_data)，
## 存放各項數據(學校/成績/交換期程/名額等等)使用，以供查詢。
### 下面開始建立一個個小DataFrame，最後再將它們全部合成新的DataFrame (search_data)裡。

### 學校資料(school_data)

In [7]:
#創立新的DataFrame: school_data
school_data = pd.DataFrame()

In [8]:
#可以先把國別、校名、甄試組別輸進來，因為他們不需要處理
school_data["國別"] = df["國別"]
school_data["校名"] = df["校名"]
school_data["甄試組別"] = df["甄試組別"]

In [9]:
school_data

,國別,校名,甄試組別
0,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組
1,馬來西亞,馬來亞大學UNIVERSITY OF MALAYA,英語組
2,馬來西亞,馬來西亞國立大學NATIONAL UNIVERSITY OF MALAYSIA,英語組
3,香港,香港教育大學THE EDUCATION UNIVERSITY OF HONG KONG,英語組
4,香港,嶺南大學LINGNAN UNIVERSITY,英語組
...,...,...,...
332,以色列,海法大學UNIVERSITY OF HAIFA,英語組
333,丹麥,南丹麥大學UNIVERSITY OF SOUTHERN DENMARK,英語組
334,丹麥,南丹麥大學UNIVERSITY OF SOUTHERN DENMARK,英語組
335,丹麥,奧胡斯大學AARHUS UNIVERSITY,英語組


### 英檢成績(eng_scores)

In [10]:
eng_scores = pd.DataFrame(data = np.zeros((len(df),3), dtype=float), columns = ["TOEFLiBT", "IELTS", "GPA"])

In [11]:
#這邊需要去觀察原資料集"語言成績要求"的input格式，還蠻複雜的，我使用re(RegEx)來尋找並擷取所需的成績

for i in range(len(eng_scores)):
    if "TOEFL" in df.語言成績要求[i]:
        s = re.search("TOEFL.*[0-9]*", df.語言成績要求[i]).group()
        toefl_score = re.search("[0-9][0-9]*", s).group()
    else:
        toefl_score = 0
        
    if "IELTS" in df.語言成績要求[i]:
        s = re.search("IELTS [0-9][0-9.]*", df.語言成績要求[i]).group()
        ielts_score = re.search("[0-9][0-9.]*", s).group()
    else:
        ielts_score = 0
    
    if "GPA" in df.語言成績要求[i]:
        s = re.search("GPA [0-9.]*/[0-9]", df.語言成績要求[i]).group()
        gpa = re.search("[0-9][0-9.]*", s).group()
    else:
        gpa = 0
        
               
    eng_scores.TOEFLiBT[i] = int(toefl_score)
    eng_scores.IELTS[i] = float(ielts_score)
    eng_scores.GPA[i] = float(gpa)

In [12]:
#TOEFL iBT成績沒有小數，所以把type轉成int
eng_scores = eng_scores.astype({"TOEFLiBT": "int32"})

In [13]:
#擷取過後的成績
eng_scores

,TOEFLiBT,IELTS,GPA
0,80,6.0,3.0
1,80,6.0,3.0
2,80,0.0,3.0
3,80,6.0,2.7
4,79,6.0,2.5
...,...,...,...
332,89,0.0,0.0
333,88,6.5,0.0
334,88,6.5,0.0
335,83,6.5,0.0


### 外語檢定/修課(foreign)

In [14]:
#若是報考外語組的同學需要看外語檢定成績and/or外語學分數
#這邊也要去觀察原資料集"語言成績要求"的input格式，使用re(RegEx)來尋找並擷取所需的成績。
school_data[school_data.甄試組別 != "英語組"]

,國別,校名,甄試組別
9,韓國,西江大學SOGANG UNIVERSITY,韓語組
12,韓國,首爾大學SEOUL NATIONAL UNIVERSITY,韓語組
13,韓國,祥明大學SANGMYUNG UNIVERSITY,韓語組
14,韓國,慶北大學KYUNGPOOK NATIONAL UNIVERSITY,韓語組
18,韓國,慶熙大學KYUNG HEE UNIVERSITY,韓語組
...,...,...,...
305,土耳其,安卡拉大學ANKARA UNIVERSITY,土語組
315,加拿大,蒙特婁大學UNIVERSITY OF MONTREAL,法語組
327,俄羅斯,烏拉爾聯邦大學URAL FEDERAL UNIVERSITY,俄語組
329,俄羅斯,聖彼得堡大學SAINT PETERSBURG STATE UNIVERSITY,俄語組


In [15]:
foreign1 = pd.DataFrame(data = np.zeros((len(df)), dtype = str), columns = ["檢定證明"])
foreign2 = pd.DataFrame(data = np.zeros((len(df),2), dtype = int), columns = ["檢定證明排序", "相關語言學分數"])
foreign = pd.concat([foreign1, foreign2], axis = 1)

In [16]:
# 我發現其實很多組別的考試要求都是檢定X級或是Y學分
for i in range(len(df)):
    
# 檢定能力: 不同組有不同的字串格式，日語組都是"日檢NX級",義語組只有一間"無資料"所以不用處理，其他組都是"...級以上之證明"
# Try: 抓到re.findall找出來的list裡面的字串， except: IndexError(list裡面沒有任何東西，代表沒找到)，定值為0

            
    if df.甄試組別[i] in ["韓語組","越語組","西語組","法語組","波語組","捷語組","德語組","俄語組"]:
        
        try:
            foreign.檢定證明[i] = re.findall("([ABC]*\d)級", df.語言成績要求[i])[0]
        except IndexError:
            foreign.檢定證明[i] = 0
            
        try:
            foreign.相關語言學分數[i] = re.findall("(\d+)學分", df.語言成績要求[i])[0]
        except IndexError:
            foreign.相關語言學分數[i] = 0
        

    
    elif df.甄試組別[i] == "日語組":
        try: 
            foreign.檢定證明[i] = re.findall("日檢(N\d)", df.語言成績要求[i])[0]
        except IndexError:
            foreign.檢定證明[i] = 0
        
        foreign.相關語言學分數[i] = 0
        
    else:
        foreign.檢定證明[i] = 0
        foreign.相關語言學分數[i] = 0
            

    
#最後再做個別成績的修正 (字串格式不符)
foreign.檢定證明[107] = "B1"

C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_

In [17]:
#接下來再依照各語言檢定的分級做順位
jp_prof = {0:0, "N5":1, "N4":2, "N3":3, "N2":4, "N1":5}
ko_prof = {0:0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6}
other_prof = {0:0, "A1":1, "A2":2, "B1":3, "B2":4, "C1":5, "C2":6}


for i in range(len(df)):
    if df.甄試組別[i] == "日語組": 
        foreign.檢定證明排序[i] = jp_prof[foreign.檢定證明[i]]
            
    elif df.甄試組別[i] == "韓語組":
        foreign.檢定證明排序[i] = ko_prof[foreign.檢定證明[i]]

    elif df.甄試組別[i] in ["越語組","西語組","法語組","德語組","波語組","俄語組"]:
        foreign.檢定證明排序[i] = other_prof[foreign.檢定證明[i]]
        
    else:
        foreign.檢定證明排序[i] = 0
        

C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\Max\Anaconda3\lib\site-packages\ipy

## 將目前有的三個DataFrame合起來之後...

In [18]:
search_data = pd.concat([school_data, foreign, eng_scores], axis = 1)
search_data[school_data.甄試組別 != "英語組"]

,國別,校名,甄試組別,檢定證明,檢定證明排序,相關語言學分數,TOEFLiBT,IELTS,GPA
9,韓國,西江大學SOGANG UNIVERSITY,韓語組,2,2,12,0,0.0,0.0
12,韓國,首爾大學SEOUL NATIONAL UNIVERSITY,韓語組,4,4,0,0,0.0,3.0
13,韓國,祥明大學SANGMYUNG UNIVERSITY,韓語組,4,4,25,0,0.0,0.0
14,韓國,慶北大學KYUNGPOOK NATIONAL UNIVERSITY,韓語組,4,4,25,0,0.0,0.0
18,韓國,慶熙大學KYUNG HEE UNIVERSITY,韓語組,3,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
305,土耳其,安卡拉大學ANKARA UNIVERSITY,土語組,0,0,0,0,0.0,0.0
315,加拿大,蒙特婁大學UNIVERSITY OF MONTREAL,法語組,B2,4,44,0,0.0,0.0
327,俄羅斯,烏拉爾聯邦大學URAL FEDERAL UNIVERSITY,俄語組,A1,1,6,0,0.0,0.0
329,俄羅斯,聖彼得堡大學SAINT PETERSBURG STATE UNIVERSITY,俄語組,A1,1,6,0,0.0,0.0


## 搜尋系統

### (v2更新)我認為搜尋時判斷條件可以依循search_data，但顯示結果時還是會顯示給使用者相對應的原始資料。

In [35]:
def search():
    
    #從完整的學校表格開始，根據搜尋結果慢慢減少
    results = df.copy()
    
    #1. 報考組別
    language_type = input("請問你這次想要報考哪個組別? ")
    results = results[results.甄試組別 == language_type]
    
    #2a. (英語組) 使用TOEFL iBT, IELTS 或 GPA 查詢
    if language_type == "英語組":
        eng_scores = {}
        english = input("請問你想要用什麼樣的英文成績查詢(可複選)? ")
    
        for i in ["TOEFL iBT", "IELTS", "GPA"]:
            if i in english:
                score = input("請問你的"+i+"成績是多少? ")
                eng_scores[i] = float(score)
            else:
                eng_scores[i] = 0
                
        results = results[(search_data.TOEFLiBT <= eng_scores["TOEFL iBT"]) & (search_data.IELTS <= eng_scores["IELTS"])\
                         & (search_data.GPA <= eng_scores["GPA"])]
    
    
    #2b. (外語組) 使用相關課程學分數及檢定級數查詢
    else:
        foreign_credits = input("請問你修了此語言相關課程共幾學分? ")
        results = results[search_data.相關語言學分數 <= int(foreign_credits)]
        
        foreign_proficiency = input("請問你擁有此語言檢定級數? (沒有請填0) ")
        
        jp_prof = {0:0, "N5":1, "N4":2, "N3":3, "N2":4, "N1":5}
        ko_prof = {0:0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6}
        other_prof = {0:0, "A1":1, "A2":2, "B1":3, "B2":4, "C1":5, "C2":6}
        
        if language_type == "日語組":
            foreign_proficiency = jp_prof[foreign_proficiency]
            
        elif language_type == "韓語組":
            foreign_proficiency = ko_prof[foreign_proficiency]
        
        else:
            foreign_proficiency = other_prof[foreign_proficiency]
        
        results = results[search_data.檢定證明排序 <= int(foreign_proficiency)]
    
    
    #3. 輸出結果
    print(f"你一共可以申請{len(results)}所學校!")
    return results

In [36]:
#實際搜尋
search()

請問你這次想要報考哪個組別? 法語組
請問你修了此語言相關課程共幾學分? 15


C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


請問你擁有此語言檢定級數? (沒有請填0) B2
你一共可以申請3所學校!


C:\Users\Max\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,國別,校名,甄試組別,限制錄取生交換出國時資格,締約學校交換申請截止時間,語言成績要求,名額,可交換開始學年度.學期（擇一）,獎學金補助
156,法國,多菲納-巴黎第九大學UNIVERSITÉ PARIS-DAUPHINE,法語組,研究所或大二以上,05-31 / 10-31,法語檢定B2級以上之證明,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料
158,法國,里昂第二大學UNIVERSITÉ LUMIÈRE LYON 2,法語組,研究所或大二以上,05-31,學生得修習法語語言相關課程12學分以上並達合格標準，或可提出法語檢定A2級以上之證明,5,109.01（一年） 109.01（一學期）,無資料
164,法國,巴黎政治大學SCIENCES PO,法語組,大三以上不含研究所,04-20 / 11-08,法語檢定B2級以上之證明,1,109.01（一年） 109.01（一學期） 109.02（一學期）,無資料


### 小工具存放區

In [ ]:
#轉成csv看output符不符合格式
search_data.to_csv("TEST.csv")

In [ ]:
#方便查詢用，甄試組別改成各組別即可
df.語言成績要求[df.甄試組別 == "韓語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "日語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "越語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "西語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "法語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "波語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "捷語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "義語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "德語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "俄語組"]

In [ ]:
df.語言成績要求[df.甄試組別 == "土語組"]